# What Makes a Champion?
## DSCI 100 Group 15 Project Proposal

Arimoto, Yuhei - Student Number | 
Dufour-Nogueira, Emma Héloïse - 34570614 | 
Huang, Felix - Student Number | 
Tan Palanca, Katrina Marie (Kim) - 98857568

### Introduction

Drawing predictions based on existing data sets provides a detailed interpretation and conclusions regarding the observations within it. Regression is one way in which predictive analysis of a data set can be achieved, predicting a numerical observation. Using the data set “Game results for Top 500 Players from 2017-2019”, specifically "atp2017-2019.csv", created by Jeff Sackman is the primary data to be manipulated.

The goal of this project is to create a K-nearest neighbors (KNN) regression model that will determine the probability that a player will win a tennis game based on their skills in serving through the following predictive question:
- How likely could a professional tennis player be to win a tennis game based on their serving skill?

### Preliminary Exploratory Data Analysis

### Methods

### Expected Outcomes